In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import os

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

This code classifies inventory items by:

First giving unique IDs to exact matches (same UOM + Class + Category + Description)

Then grouping similar descriptions together using text clustering

Result: Every item gets an ID, with similar items sharing the same ID

Prints all items grouped by their final IDs

Saves results to Excel file

In [4]:
class ItemClassifier:
    def __init__(self, eps=0.2, min_samples=2):
        self.eps = eps
        self.min_samples = min_samples

    def preprocess_text(self, text):
        """Clean and normalize text for similarity comparison"""
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text)
        words = text.split()
        return ' '.join(sorted(words))

    def load_and_validate_data(self, file_path):
        """Load Excel file and validate required columns"""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File '{file_path}' not found")

        df = pd.read_excel(file_path)
        df.columns = df.columns.str.strip()

        required_columns = [
            'Item Primary UOM', 'Item Class', 'Category',
            'Item', 'Item Desc'
        ]
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        return df

    def assign_exact_match_ids(self, df):
        """Assign IDs to items with exact matches"""
        df['clean_desc'] = df['Item Desc'].apply(self.preprocess_text)
        df['group_key'] = df['Item Primary UOM'] + '|' + df['Item Class'] + '|' + df['Category']
        df['Item ID'] = 0

        unique_items = df.drop_duplicates(
            subset=['Item Primary UOM', 'Item Class', 'Category', 'clean_desc']
        )

        current_id = 1
        for _, row in unique_items.iterrows():
            mask = (
                (df['Item Primary UOM'] == row['Item Primary UOM']) &
                (df['Item Class'] == row['Item Class']) &
                (df['Category'] == row['Category']) &
                (df['clean_desc'] == row['clean_desc'])
            )
            df.loc[mask, 'Item ID'] = current_id
            current_id += 1

        return df, current_id

    def group_similar_items(self, df):
        """Group similar items using DBSCAN clustering"""
        for group_key in df['group_key'].unique():
            group_mask = df['group_key'] == group_key
            group_df = df[group_mask].copy()
            group_texts = group_df['clean_desc'].tolist()

            if len(group_texts) > 1:
                vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
                X = vectorizer.fit_transform(group_texts)

                dbscan = DBSCAN(
                    eps=self.eps,
                    min_samples=self.min_samples,
                    metric='cosine'
                )
                clusters = dbscan.fit_predict(X.toarray())

                cluster_mapping = {}
                for i, cluster in enumerate(clusters):
                    if cluster != -1:
                        item_idx = group_df.index[i]
                        original_id = df.at[item_idx, 'Item ID']

                        if cluster not in cluster_mapping:
                            cluster_mapping[cluster] = original_id
                        else:
                            df.at[item_idx, 'Item ID'] = cluster_mapping[cluster]

        return df

    def classify_items(self, file_path):
        """Main classification pipeline"""
        df = self.load_and_validate_data(file_path)
        df, _ = self.assign_exact_match_ids(df)
        df = self.group_similar_items(df)

        # Clean up temporary columns
        df = df.drop(['clean_desc', 'group_key'], axis=1, errors='ignore')
        return df


def print_results(result_df):
    """Print formatted results"""
    print("=" * 60)
    print("ITEM CLASSIFICATION RESULTS")
    print("=" * 60)

    for item_id in sorted(result_df['Item ID'].unique()):
        cluster_items = result_df[result_df['Item ID'] == item_id]
        print(f"\n📦 ITEM GROUP {item_id} ({len(cluster_items)} items):")
        print("-" * 50)

        for _, row in cluster_items.iterrows():
            print(f"   • {row['Item']} - {row['Item Desc']}")


def main():
    file_path = "itemdetails report_items.xlsx"
    classifier = ItemClassifier(eps=0.2, min_samples=2)

    try:
        print("Starting item classification...")
        result_df = classifier.classify_items(file_path)

        print_results(result_df)

        # Save results
        result_df.to_excel("Classified items.xlsx", index=False)

        print("\n" + "=" * 60)
        print(f"Results saved to: Classified item.xlsx")
        print(f"Total unique item groups: {len(result_df['Item ID'].unique()):,}")
        print(f"Total items processed: {len(result_df):,}")
        print("=" * 60)

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except ValueError as e:
        print(f"Validation Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")


if __name__ == "__main__":
    main()

Starting item classification...
ITEM CLASSIFICATION RESULTS

📦 ITEM GROUP 0 (1 items):
--------------------------------------------------
   • nan - nan

📦 ITEM GROUP 1 (1 items):
--------------------------------------------------
   • 001436 - FITTING 6MM

📦 ITEM GROUP 2 (1 items):
--------------------------------------------------
   • 01-010-0004 - CYLINDER BLOCK

📦 ITEM GROUP 3 (1 items):
--------------------------------------------------
   • 0119811305 - BEARING 6205ZZC3

📦 ITEM GROUP 4 (1 items):
--------------------------------------------------
   • 02-020-0022 - VALVE PLATE

📦 ITEM GROUP 5 (1 items):
--------------------------------------------------
   • 02-023-0004 - RET PLATE

📦 ITEM GROUP 6 (1 items):
--------------------------------------------------
   • 020-1053 - GSM ANTENNA AND GPS ANTENNA

📦 ITEM GROUP 7 (8 items):
--------------------------------------------------
   • 02051604 - AIR FILTER OUTER
   • 1209590 - AIR FILTER OUTER
   • 191526 - AIR FILTER OUTER
   • 1